In [2]:
import tensorflow as tf
from keras.datasets import mnist
from keras.layers import Dense, Flatten, Reshape, Input, BatchNormalization
from keras.layers.activation import LeakyReLU
from keras.models import Sequential, Model
from keras.optimizers import Adam
import numpy as np


c:\Users\Shaunak\anaconda3\envs\py310\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
img_r = 28
img_c = 28
channels = 1
image_shape = (img_r, img_c, channels)

In [4]:
def generator():
    noise_shape = (100,)

    model = Sequential()
    model.add(Dense(256, input_shape = noise_shape))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum = 0.8))
    model.add(Dense(512, input_shape = noise_shape))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum = 0.8))
    model.add(Dense(1024, input_shape = noise_shape))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum = 0.8))
    model.add(Dense(np.prod(image_shape), activation='tanh'))
    model.add(Reshape(image_shape))

    noise = Input(shape=noise_shape)
    img = model(noise)

    return Model(noise, img)

In [5]:
def discriminator():

    model = Sequential()
    model.add(Flatten(input_shape=image_shape))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(1, activation = 'sigmoid'))

    img = Input(shape=image_shape)
    validity = model(img)

    return Model(img, validity)

In [6]:
def train(epochs, batch_size=128, save_interval=500):
    (x_train, _), (_, _) = mnist.load_data()

    x_train = (x_train.astype(np.float32) - 127.5) / 127.5

    x_train = np.expand_dims(x_train, axis=3)

    half_batch = int(batch_size/2)


In [7]:
(x_train, x_test), (y_train, y_test) = mnist.load_data()

print(len(x_train))

60000
